# Examples for Using HHL

## Simple Example to Validate Result

Let's first try a simple example and validate that our code gives the correct results. Remember that the answer in the HHL algorithm is not classical, but encoded in a statevector, which is difficult to directly observe. In order to validate our result, we use the `Statevector` class from the `quantum_info` package to obtain the exact vector components.

In [1]:
import numpy as np

from hhl import HHL
from qiskit.quantum_info import Statevector

A = np.array([[1, -1/3], [-1/3, 1]])
b = np.array([1, 0])

qc = HHL(A, b)

print("quantum circuit: ")
print(qc)

quantum_answer = Statevector(qc).data.real[16:18] / qc.scaling # 16 = 10000; 17 = 10001
classic_answer = np.linalg.solve(A,b)

np.set_printoptions(3)

print("quantum answer = ", quantum_answer)
print("classic answer = ", classic_answer)

quantum circuit: 
      ┌─────────────┐┌──────┐        ┌─────────┐
  q0: ┤ circuit-161 ├┤3     ├────────┤3        ├
      └─────────────┘│      │┌──────┐│         │
q1_0: ───────────────┤0     ├┤2     ├┤0        ├
                     │  QPE ││      ││  QPE_dg │
q1_1: ───────────────┤1     ├┤1     ├┤1        ├
                     │      ││  1/x ││         │
q1_2: ───────────────┤2     ├┤0     ├┤2        ├
                     └──────┘│      │└─────────┘
  q2: ───────────────────────┤3     ├───────────
                             └──────┘           
quantum answer =  [1.125 0.375]
classic answer =  [1.125 0.375]


## Analyse Circuit Complexity

Let's first look empirically how deep is our circuit.

In [2]:
from qiskit import transpile
from scipy.sparse import diags
from pprint import pprint


def transpiled_depth(qc):
    basis_gates = ["u1", "u2", "u3", "cx", "id"]
    qc_transpiled = transpile(qc, basis_gates=basis_gates)
    return qc_transpiled.depth()

MAX_QUBITS = 4

depths = []
for n_qubits in range(1, MAX_QUBITS + 1):
    # construct sparser matrix
    A_ = diags(
        [-1 / 3, 1, -1 / 3], [-1, 0, 1], shape=(2**n_qubits, 2**n_qubits)
    ).toarray()
    b_ = np.array([1] + [0] * (2**n_qubits - 1))

    qc = HHL(A_, b_)

    qc_state_prepare = qc._get_state_preparation()
    qc_H_simulation = qc._get_ejHt()
    qc_QPE = qc._get_QPE()
    qc_controled_rotation = qc._get_controld_rotation()
    qc_QPE_inverse = qc._get_QPE().inverse()

    depths.append(
        {
            "total depth: ": [
                transpiled_depth(qc),
                {
                    "state prepare: ": transpiled_depth(qc_state_prepare),
                    "QPE: ": [
                        transpiled_depth(qc_QPE),
                        {"H simulation: ": transpiled_depth(qc_H_simulation)},
                    ],
                    "controled rotation: ": transpiled_depth(qc_controled_rotation),
                    "QPE inverse: ": transpiled_depth(qc_QPE_inverse),
                },
            ],
            "num clock qubits: ": qc.num_clock_qubits,
            "evolution_time" : qc.evolution_time
        }
    )

pprint(depths)


[{'evolution_time': 1.1780972450961722,
  'num clock qubits: ': 3,
  'total depth: ': [199,
                    {'QPE inverse: ': 41,
                     'QPE: ': [41, {'H simulation: ': 1}],
                     'controled rotation: ': 120,
                     'state prepare: ': 0}]},
 {'evolution_time': 1.7049583499242287,
  'num clock qubits: ': 4,
  'total depth: ': [2803,
                    {'QPE inverse: ': 1283,
                     'QPE: ': [1287, {'H simulation: ': 9}],
                     'controled rotation: ': 240,
                     'state prepare: ': 0}]},
 {'evolution_time': 1.5769432397293481,
  'num clock qubits: ': 5,
  'total depth: ': [24791,
                    {'QPE inverse: ': 12174,
                     'QPE: ': [12148, {'H simulation: ': 33}],
                     'controled rotation: ': 480,
                     'state prepare: ': 0}]},
 {'evolution_time': 1.708949637401716,
  'num clock qubits: ': 6,
  'total depth: ': [156139,
                    {'QPE

We see that the most circuit depth is from QPE, so let's look into it. QPE mainly consist of two subroutines: controled unitary operations and QFT dagger. We can't directly get the subroutines from QPE API, so I copied the code here and modify it to test the QFT circuit depth for 4 qubits' input. Let's see what will happen. 

In [3]:
from typing import Optional

from qiskit.circuit import QuantumCircuit, QuantumRegister

from qiskit.circuit.library import QFT
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate

# debug
basis_gates = ["u1", "u2", "u3", "cx", "id"]

# code copied from qiskit
class PhaseEstimation(QuantumCircuit):
    def __init__(
        self,
        num_evaluation_qubits: int,
        unitary: QuantumCircuit,
        iqft: Optional[QuantumCircuit] = None,
        name: str = "QPE",
    ) -> None:

        qr_eval = QuantumRegister(num_evaluation_qubits, "eval")
        qr_state = QuantumRegister(unitary.num_qubits, "q")
        circuit = QuantumCircuit(qr_eval, qr_state, name=name)

        if iqft is None:
            iqft = QFT(num_evaluation_qubits, inverse=True, do_swaps=False).reverse_bits()

        circuit.h(qr_eval)  # hadamards on evaluation qubits

        for j in range(num_evaluation_qubits):  # controlled powers
            # debug, my code for output circuit depth of the components
            upj = unitary.power(2**j)
            upj_control = upj.control()

            print("unitary.power2**", j ," depth = ", transpiled_depth(upj))
            print("unitary.power2**", j ,".control() depth = ", transpiled_depth(upj_control))
            
            circuit.compose(upj_control, qubits=[j] + qr_state[:], inplace=True)

        circuit.compose(iqft, qubits=qr_eval[:], inplace=True)  # final QFT

        super().__init__(*circuit.qregs, name=circuit.name)
        self.compose(circuit.to_gate(), qubits=self.qubits, inplace=True)

        # debug, my code for output circuit depth of the components
        controled_us_transpiled = transpile(circuit, basis_gates=basis_gates)
        print("us depth = ", controled_us_transpiled.depth())

        iqft_transpiled = transpile(iqft, basis_gates=basis_gates)
        print("iqft depth: ", iqft_transpiled.depth())


# code only changed the parameter
#
def _get_ejHt(matrix, evolution_time, num_vec_qubits):
    op_H = SparsePauliOp.from_operator(matrix)
    op_ejHt = PauliEvolutionGate(op_H, -evolution_time)
    qc_ejHt = QuantumCircuit(num_vec_qubits)
    qc_ejHt.append(op_ejHt, range(int(num_vec_qubits)))
    return qc_ejHt
#
def _get_QPE(num_clock_qubits, matrix, evolution_time, num_vec_qubits):
    qc_ejHt = _get_ejHt(matrix,evolution_time, num_vec_qubits)
    qc_pe_ejHt = PhaseEstimation(num_clock_qubits, qc_ejHt)
    return qc_pe_ejHt

# We input exactly the same parameter

MAX_QUBITS = 4

depths = []

i = 0
# previously recorded
nums_clock_qubits = [3,4,5,6]
evolution_times = np.array([1.1780972450961722, 1.7049583499242287,1.5769432397293481,1.708949637401716])

for n_qubits in range(1, MAX_QUBITS + 1):
    # construct sparser matrix
    A_ = diags(
        [-1 / 3, 1, -1 / 3], [-1, 0, 1], shape=(2**n_qubits, 2**n_qubits)
    ).toarray()
    # b_ = np.array([1] + [0] * (2**n_qubits - 1))
    _ = _get_QPE(nums_clock_qubits[i], A_, evolution_times[i], n_qubits)
    i += 1


unitary.power2** 0  depth =  1
unitary.power2** 0 .control() depth =  5
unitary.power2** 1  depth =  1
unitary.power2** 1 .control() depth =  9
unitary.power2** 2  depth =  1
unitary.power2** 2 .control() depth =  17
us depth =  41
iqft depth:  13
unitary.power2** 0  depth =  9
unitary.power2** 0 .control() depth =  86
unitary.power2** 1  depth =  17
unitary.power2** 1 .control() depth =  171
unitary.power2** 2  depth =  33
unitary.power2** 2 .control() depth =  341
unitary.power2** 3  depth =  65
unitary.power2** 3 .control() depth =  681
us depth =  1287
iqft depth:  21
unitary.power2** 0  depth =  33
unitary.power2** 0 .control() depth =  394
unitary.power2** 1  depth =  65
unitary.power2** 1 .control() depth =  787
unitary.power2** 2  depth =  129
unitary.power2** 2 .control() depth =  1573
unitary.power2** 3  depth =  257
unitary.power2** 3 .control() depth =  3145
unitary.power2** 4  depth =  513
unitary.power2** 4 .control() depth =  6289
us depth =  12148
iqft depth:  29
unitar

From the above results, we observe that the majority of the circuit depth comes from controlled unitary operations. These unitaries are obtained from Hamiltonian simulation, which is known to have exponential complexity according to the Qiskit documentation. However, the exponential depth growth for Hamiltonian simulation is not obvious in our case (1, 9, 33, 89) because we selected a simple matrix for the simulation.

Despite choosing a simple matrix, the resulting circuit is still too deep for current quantum computers. That's because: 
1. Quantum Phase Estimation (QPE) requires repeatedly applying controlled $ U^{2^n} $ for n times, where n equals to the number of clock qubits. 
2. Controlled $ U $ requires approximately 10 times the resources of $ U $.

# Detailed Analysis

The implementation of our HHL algorithm relies on existing APIs, including state_prepare, Hamiltonian_simulation, trotter_suzuki, QPE, and controlled_rotation. While these APIs significantly reduce the complexity of implementing algorithms by allowing us to work with high-level concepts, they may obscure our understanding of the detailed construction of the circuit.

In this analysis, we delve into the APIs to understand the detailed construction of the circuit. Our goal is to establish a solid connection between the complexity analysis presented in the paper and the circuit complexity in our implementation.